# ONS Register of Geographic Codes

The  [*ONS Register of Geographic Codes*](http://geoportal.statistics.gov.uk/datasets?q=Register%20of%20Geographic%20Codes&sort=-updatedAt) is a register maintained by the Office of National Statistics (ONS) that describes a range of coding schemes for various UK geographies.

*(The above link should show most recent codes file at the top, but you might find the most recent file is somewhere down the list).*

The register is provided as an Excel spreadsheet containing multiple worksheets, with each worksheet contianing the codes for a particular geography.

This notebook parses the codes into a set of unnormalised relational database tables in a SQLite3 database.

__See also:__ *GSS blog* - [Why Do We Need Another Register?](https://gss.civilservice.gov.uk/blog/2017/09/need-another-register/), which describes the development of a new set of official registers for describing UK geographies.

## Accessing the Data

The *ONS Register of Geographic Codes* is updated several times a year. Finf the most current version [here](https://ons.maps.arcgis.com/home/search.html?t=content&q=tags%3ARegister%20of%20Geographic%20Codes&start=1&sortOrder=desc&sortField=relevance).

In [1]:
import sqlite3
con = sqlite3.connect("onsgeocodes.sqlite")

In [2]:
import pandas as pd

In [3]:
#Create a function to grab a zip file from an online location and then grab a specified file from inside it
import requests, zipfile

#The following fudge copes with Python 2 and Python 3
try:
    from StringIO import StringIO as zreader
except ImportError:
    from io import BytesIO as zreader

def ziparchivereader(f):
    return zipfile.ZipFile(f, 'r')

def zipgrabber(url):
    ''' Grab a zip file from a URL '''
    r = requests.get(url)
    z = ziparchivereader(zreader(r.content))
    #z = zipfile.ZipFile(zreader(r.content))
    return z

def zipgrabberfile(url, f):
    ''' Grab a file by name from a zip file grabbed from a URL '''
    return zipgrabber(url).open(f)

def zipfilebyname(z,f):
    ''' Grab a file by name from an already grabbed zip file '''
    return z.open(f)

def zipfilelist(z):
    ''' Return the names of files contained in a grabbed zip file '''
    return z.namelist()

In [4]:
url="https://ons.maps.arcgis.com/sharing/rest/content/items/af08a65b733e4ea2bc710b9ef0ed33db/data"

!mkdir -p downloads
f='downloads/tmp_register_ons_geo_codes.zip'
!rm {f}
!wget -O {f} {url}

rm: cannot remove 'downloads/tmp_register_ons_geo_codes.zip': No such file or directory
--2019-02-28 16:16:09--  https://ons.maps.arcgis.com/sharing/rest/content/items/af08a65b733e4ea2bc710b9ef0ed33db/data
Resolving ons.maps.arcgis.com (ons.maps.arcgis.com)... 54.210.197.173, 52.207.151.14, 54.86.254.142
Connecting to ons.maps.arcgis.com (ons.maps.arcgis.com)|54.210.197.173|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ago-item-storage.s3-external-1.amazonaws.com/af08a65b733e4ea2bc710b9ef0ed33db/Register_of_Geographic_Codes_%28December_2018%29_UK.zip?X-Amz-Security-Token=FQoGZXIvYXdzEOn%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDK65mMLes3A625%2F%2FGSK3A1uwZxsUtY1pgbJmSIPEL6M9i%2FmTAqC6thFW2s4CMQGDqArNsonvq4T58lqGRKAHtf0Gz%2B9PEJHQodkhdA983M8bjGpgewFxvAoGQVIX2q91DAvWU4SqsGd%2FDNuPmwxDEioue279Nodmz7yTyxAjUU6J2m4cDHeMn3ILbrbW9mowRq%2B9mB%2Bj5rHuMkjAxLsrYcWd6D9%2F7AWwlUgNQcrKQrffj9V8vSyda9ruriAzFn%2Brab6lPXQBpUPFWCYtKloWfr4wt68pbhYvVFIV5ulmDD84fPPUjNdoq49mhbi

In [5]:
zf=ziparchivereader(f)
zipfilelist(zf)

['RGC_DEC_2018_UK.csv', 'RGC_DEC_2018_UK.xlsx']

The `.csv` file contains metadata describing the geographies listed in separate sheets in the `.xlsx` file.

In [9]:
#Hacky - should really search and pull out the .csv file based on suffix
datafilestub = zipfilelist(zf)[0].split('.')[0]

metadata=pd.read_csv(zf.open('{}.csv'.format(datafilestub)))
metadata.head()

,Entity code,Entity name,Entity abbreviation,Entity theme,Entity coverage,Related entity codes,Status,Number of live instances,Number of archived instances,Number of cross-border instances,Date of last instance change,Current code (first in range),Current code (last in range),Reserved code (for CHD use),Entity owner,Date entity introduced on RGC,Entity start date
0,E92,Country,CTRY,Administrative,England,"W92, S92, N92, L93, M83",Current,1,0,0,NaN,E92000001,E92000001,E92999999,ONS,01/01/2009,01/01/2009
1,E00,Output Areas,OA,Statistical Building Block,England,"W00, S00, N00",Current,"171,372","7,582",0,31/12/2011,E00000001,E00176774,E00999999,ONS,01/01/2009,01/01/2009
2,E01,"Super Output Areas, Lower Layer",LSOA,Statistical Building Block,England,"W01, S01",Current,"32,844","1,357",0,31/12/2011,E01000001,E01033768,E01999999,ONS,01/01/2009,01/01/2003
3,E02,"Super Output Areas, Middle Layer",MSOA,Statistical Building Block,England,"W02, S02",Current,"6,791",141,0,31/12/2011,E02000001,E02006934,E02999999,ONS,01/01/2009,01/01/2003
4,E04,Civil Parishes,PAR,Administrative,England,NaN,Current,"10,460","2,364",0,17/10/2018,E04000001,E04012897,E04999999,MHCLG,01/01/2009,01/01/2009


## Extracting Geography Codes

In [13]:
#This may take some time to run...
xl=pd.read_excel(zf.open('{}.xlsx'.format(datafilestub)), sheetname=None)
xl.keys()

odict_keys(['RGC', 'Metadata_for_geography_listings', 'E00_OA', 'E01_LSOA', 'E02_MSOA', 'E04_PAR', 'E05_WD', 'E06_UA', 'E07_NMD', 'E08_MD', 'E09_LONB', 'E10_CTY', 'E11_MCTY', 'E12_RGN', 'E13_IOL', 'E14_WPC', 'E15_EER', 'E20_CANREG', 'E22_CSP', 'E23_PFA', 'E25_PUA', 'E26_NPARK', 'E28_REGD', 'E29_REGSD', 'E30_TTWA', 'E31_FRA', 'E32_LAC', 'E33_WZ', 'E34_BUA', 'E35_BUASD', 'E36_CMWD', 'E37_LEP', 'E38_CCG', 'E39_NHSRLO', 'E40_NHSCR', 'E41_CMLAD', 'E42_CMCTY', 'E43_NCP', 'E45_PHEC', 'E46_PHEREG', 'E47_CAUTH', 'E48_LRF', 'E49_EZ', 'E50_WA', 'E51_DC', 'E52_LEPOP', 'E53_LEPNOP', 'E54_STP', 'E55_SCN', 'E56_CAL', 'E57_NCV', 'E58_CED', 'E59_ICS', 'E92_CTRY', 'J01_TCITY', 'J02_PAR', 'J03_WD', 'J04_LAD', 'J05_CTY', 'K01_TTWA', 'K02_UK', 'K03_GB', 'K04_EW', 'K05_BUA', 'K06_BUASD', 'L00_SHA', 'L93_BCD', 'M00_SHA', 'M01_PHD', 'M83_BCD', 'N00_SA', 'N06_WPC', 'N07_EER', 'N08_WD', 'N09_LGD', 'N10_DEA', 'N11_SETT2015', 'N12_TTWA', 'N19_WZ', 'N23_PFA', 'N24_PFD', 'N31_NIFRS', 'N32_NIFRSA', 'N33_NIFRSD', 'N9

The register appears to describe only a subset of the codesets listed in the metadata file - codes relating specifically to Scotland are not provided.

In [14]:
metadata["codeAbbrv"] = metadata["Entity code"].map(str) + '_' + metadata["Entity abbreviation"].map(str)
codes = metadata["codeAbbrv"].tolist()
print(set(codes) - set(xl.keys()))

{'S25_CJA', 'W16_DCELL', 'S40_SFRSDA', 'S09_ER', 'E17_CT', 'S39_SFRLSO', 'S23_PFA', 'W38_BUASD', 'S01_DZ', 'S10_URC', 'E40_NHSER', 'S17_SPR', 'S03_CHP', 'S20_SETT', 'S15_EER', 'S24_HIE', 'S41_SMR', 'S32_SPD', 'S29_CLC', 'K04_E&W', 'S22_TTWA', 'S33_BRMA', 'S21_NPARK', 'S26_CHCP', 'S27_ISDT', 'S02_IZ', 'E19_PSHA', 'S06_ROAL', 'S19_LOC', 'S35_CVP', 'S05_ROAC', 'E27_NDC', 'S36_ISLG', 'S31_LAU2', 'S34_WZ', 'S04_ROAS', 'S11_SDPA', 'S13_WD', 'S38_SFRS', 'S37_HIA', 'E16_PCT', 'E18_SHA', 'S00_OA', 'S28_CDC', 'S12_CA', 'S92_CTRY', 'S07_RTP', 'S14_WPC', 'S16_SPC', 'S30_LAU1', 'E21_CANNET', 'S08_HB', 'E24_LLSC'}


Metadata for the spreadsheet is contained in the *RGC* sheet.

In [15]:
xl['RGC']["codeAbbrv"] = xl['RGC']["Entity code"].map(str) + '_' + xl['RGC']["Entity abbreviation"].map(str)

xl['RGC'].to_sql(con=con, name='metadata', index=False, if_exists='replace')

/srv/conda/lib/python3.6/site-packages/pandas/core/generic.py:2532: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [16]:
cols=['GEOGCD','GEOGNM','GEOGNMW','OPER_DATE','TERM_DATE','STATUS']

bigcodes=pd.DataFrame(columns=['sheet']+cols)
bigcodes.to_sql(con=con, name='codelist', index=False, if_exists='replace')

sheets= list(xl.keys())
sheets.remove('For_Scotland')
for sheet in sheets[2:]:
    xl[sheet].to_sql(con=con, name=sheet, index=False, if_exists='replace')
    xl[sheet]['sheet']=sheet
    #Reorder the columns
    xl[sheet][['sheet']+cols].to_sql(con=con, name='codelist', index=False, if_exists='append')

In [17]:
q='SELECT * FROM E02_MSOA LIMIT 5'
pd.read_sql_query(q, con)

,GEOGCD,GEOGNM,GEOGNMW,SI_ID,SI_TITLE,OPER_DATE,TERM_DATE,PARENTCD,ENTITYCD,OWNER,STATUS,AREAEHECT,AREACHECT,AREAIHECT,AREALHECT
0,E02000001,City of London 001,None,None,None,2004-08-01 00:00:00,None,None,E02,ONS,live,315.14,289.78,0.0,289.78
1,E02000002,Barking and Dagenham 001,None,None,None,2004-08-01 00:00:00,None,None,E02,ONS,live,216.15,216.15,0.0,216.15
2,E02000003,Barking and Dagenham 002,None,None,None,2004-08-01 00:00:00,None,None,E02,ONS,live,214.15,214.15,0.0,214.15
3,E02000004,Barking and Dagenham 003,None,None,None,2004-08-01 00:00:00,None,None,E02,ONS,live,249.28,249.28,0.0,249.28
4,E02000005,Barking and Dagenham 004,None,None,None,2004-08-01 00:00:00,None,None,E02,ONS,live,118.81,118.81,0.0,118.81


In [18]:
q='SELECT * FROM codelist WHERE "GEOGCD"="{code}"'.format(code='W40000004')
pd.read_sql_query(q, con)

,sheet,GEOGCD,GEOGNM,GEOGNMW,OPER_DATE,TERM_DATE,STATUS
0,W40_CMLAD,W40000004,Denbighshire,Sir Ddinbych,2011-12-31 00:00:00,None,live


In [19]:
q='''
SELECT *  FROM codelist JOIN metadata 
WHERE "GEOGNM"="{name}" AND codeAbbrv=sheet AND codelist.STATUS="live"
'''.format(name='Isle of Wight')
pd.read_sql_query(q, con)

,sheet,GEOGCD,GEOGNM,GEOGNMW,OPER_DATE,TERM_DATE,STATUS,Entity code,Entity name,Entity abbreviation,...,Number of archived instances,Number of cross-border instances,Date of last instance change,Current code (first in range),Current code (last in range),Reserved code (for CHD use),Entity owner,Date entity introduced on RGC,Entity start date,codeAbbrv
0,E06_UA,E06000046,Isle of Wight,None,2009-01-01 00:00:00,None,live,E06,Unitary Authorities,UA,...,1,0,2013-04-01 00:00:00,E06000001,E06000057,E06999999,MHCLG,2009-01-01 00:00:00,2009-01-01 00:00:00,E06_UA
1,E14_WPC,E14000762,Isle of Wight,None,2010-05-06 00:00:00,None,live,E14,Westminster Parliamentary Constituencies,WPC,...,529,0,2010-05-06 00:00:00,E14000530,E14001062,E14999999,LGBC,2009-01-01 00:00:00,2009-01-01 00:00:00,E14_WPC
2,E22_CSP,E22000116,Isle of Wight,None,2009-01-01 00:00:00,None,live,E22,Community Safety Partnerships,CSP,...,81,0,2016-04-01 00:00:00,E22000001,E22000374,E22999999,Home Office,2009-01-01 00:00:00,2009-01-01 00:00:00,E22_CSP
3,E28_REGD,E28000146,Isle of Wight,None,2009-01-01 00:00:00,None,live,E28,Registration Districts,REGD,...,67,0,2013-01-17 00:00:00,E28000001,E28000219,E28999999,ONS,2009-01-01 00:00:00,2009-01-01 00:00:00,E28_REGD
4,E30_TTWA,E30000070,Isle of Wight,None,2009-01-01 00:00:00,None,live,E30,Travel to Work Areas,TTWA,...,206,6,2015-07-31 00:00:00,E30000004,E30000294,E30999999,ONS,2009-01-01 00:00:00,2009-01-01 00:00:00,E30_TTWA
5,E31_FRA,E31000021,Isle of Wight,None,2009-01-01 00:00:00,None,live,E31,Fire and Rescue Authorities,FRA,...,10,0,2016-08-01 00:00:00,E31000001,E31000047,E31999999,MHCLG,2009-01-01 00:00:00,2009-01-01 00:00:00,E31_FRA
6,E41_CMLAD,E41000046,Isle of Wight,None,2011-12-31 00:00:00,None,live,E41,Census Merged Local Authority Districts,CMLAD,...,0,0,None,E41000001,E41000324,E41999999,ONS,2013-04-01 00:00:00,2011-12-31 00:00:00,E41_CMLAD


In [20]:
q='''
SELECT DISTINCT "Entity name", sheet  FROM codelist JOIN metadata 
WHERE "GEOGNM" LIKE "%{name}%" AND codeAbbrv=sheet AND codelist.STATUS="live"
'''.format(name='Isle of Wight')
pd.read_sql_query(q, con)

,Entity name,sheet
0,"Super Output Areas, Lower Layer",E01_LSOA
1,"Super Output Areas, Middle Layer",E02_MSOA
2,Unitary Authorities,E06_UA
3,Westminster Parliamentary Constituencies,E14_WPC
4,Community Safety Partnerships,E22_CSP
5,Registration Districts,E28_REGD
6,Registration Sub-district,E29_REGSD
7,Travel to Work Areas,E30_TTWA
8,Fire and Rescue Authorities,E31_FRA
9,Built-up Areas,E34_BUA


In [21]:
#Lookup a code
q='''
SELECT * FROM codelist JOIN metadata 
WHERE "GEOGCD" = "{name}" AND codeAbbrv=sheet 
'''.format(name='E05008479')
pd.read_sql_query(q, con)

,sheet,GEOGCD,GEOGNM,GEOGNMW,OPER_DATE,TERM_DATE,STATUS,Entity code,Entity name,Entity abbreviation,...,Number of archived instances,Number of cross-border instances,Date of last instance change,Current code (first in range),Current code (last in range),Reserved code (for CHD use),Entity owner,Date entity introduced on RGC,Entity start date,codeAbbrv
0,E05_WD,E05008479,Arreton and Newchurch,None,2009-06-04 00:00:00,None,live,E05,Electoral Wards/Divisions,WD,...,4294,0,2018-05-03 00:00:00,E05000026,E05011549,E05999999,MHCLG,2009-01-01 00:00:00,2009-01-01 00:00:00,E05_WD


## Creating a Simple API

There are a couple of ways we can create a simple service via datasette.

In [45]:
import requests, json
#import pandas as pd
import geopandas as gpd
from shapely.geometry import shape 


def query_datasette_api(q,db_url = "http://localhost:8001/adminboundaries.json",
                        dataframe=False, geojsoncol=None):
    ''' Simple query to datasette endpoint. Return response as a dict or pandas dataframe. '''
    params = {'sql': q}

    r = requests.get(db_url, params=params)
    jdata=r.json()
    if dataframe:
        df=gpd.GeoDataFrame(jdata['rows'])
        df.columns = jdata['columns']
        for c in df.columns:
            #Need a better way to identify geo columns?
            #Also, this should really be a geopandas dataframe, with the json column as a geometry column
            if c.startswith('AsGeoJSON'):
                df[c]=df[c].apply(json.loads)
                cn=c.replace('AsGeoJSON(','').replace(')','')
                df[cn]=df[c].apply(shape)
        return df
    
    #Need a better way to identify geo columns?
    ix = [jdata['columns'].index(c) for c in jdata['columns'] if c.startswith('AsGeoJSON')]
    for i in ix:
        for i2 in range(len(jdata['rows'])):
            jdata['rows'][i2][i]=json.loads(jdata['rows'][i2][i])

    geojsoncol  = geojsoncol if geojsoncol and geojsoncol.startswith('AsGeoJSON') else 'AsGeoJSON({})'.format(geojsoncol)
    if geojsoncol and geojsoncol in jdata['columns']:
        return jdata['rows'][jdata['columns'].index(geojsoncol)][0]
    return jdata

In [28]:
#Datasette currently requires specific version of pip?
#!pip install --upgrade click==6.7

# Example of running datasette server from inside a code cell
# via https://nbviewer.jupyter.org/gist/minrk/622080cf8af787734805d12bec4ae302from threading import Thread
from threading import Thread

def app_in_thread():
    ! datasette onsgeocodes.sqlite --load-extension=/usr/lib/x86_64-linux-gnu/mod_spatialite.so --config sql_time_limit_ms:10000 --cors

t = Thread(target=app_in_thread)
t.start()

Serve! files=('onsgeocodes.sqlite',) on port 8001
[2019-02-28 16:26:30 +0000] [114] [INFO] Goin' Fast @ http://127.0.0.1:8001
[2019-02-28 16:26:30 +0000] [114] [INFO] Starting worker [114]


In [46]:
area_name = 'Isle of Wight'

In [48]:
q='''
SELECT * FROM codelist JOIN metadata 
WHERE "GEOGNM"="{name}" AND codeAbbrv=sheet AND codelist.STATUS="live"'''

tmp = query_datasette_api(q.format(name = area_name),
                          "http://localhost:8001/onsgeocodes.json", dataframe=True)
tmp

[2019-02-28 16:35:32 +0000] - (sanic.access)[INFO][1:2]: GET http://localhost:8001/onsgeocodes.json?sql=%0ASELECT+%2A+FROM+codelist+JOIN+metadata+%0AWHERE+%22GEOGNM%22%3D%22Isle+of+Wight%22+AND+codeAbbrv%3Dsheet+AND+codelist.STATUS%3D%22live%22  302 0
[2019-02-28 16:35:32 +0000] - (sanic.access)[INFO][1:2]: GET http://localhost:8001/onsgeocodes-93c9084.json?sql=%0ASELECT+%2A+FROM+codelist+JOIN+metadata+%0AWHERE+%22GEOGNM%22%3D%22Isle+of+Wight%22+AND+codeAbbrv%3Dsheet+AND+codelist.STATUS%3D%22live%22  200 2973


,sheet,GEOGCD,GEOGNM,GEOGNMW,OPER_DATE,TERM_DATE,STATUS,Entity code,Entity name,Entity abbreviation,...,Number of archived instances,Number of cross-border instances,Date of last instance change,Current code (first in range),Current code (last in range),Reserved code (for CHD use),Entity owner,Date entity introduced on RGC,Entity start date,codeAbbrv
0,E06_UA,E06000046,Isle of Wight,None,2009-01-01 00:00:00,None,live,E06,Unitary Authorities,UA,...,1,0,2013-04-01 00:00:00,E06000001,E06000057,E06999999,MHCLG,2009-01-01 00:00:00,2009-01-01 00:00:00,E06_UA
1,E14_WPC,E14000762,Isle of Wight,None,2010-05-06 00:00:00,None,live,E14,Westminster Parliamentary Constituencies,WPC,...,529,0,2010-05-06 00:00:00,E14000530,E14001062,E14999999,LGBC,2009-01-01 00:00:00,2009-01-01 00:00:00,E14_WPC
2,E22_CSP,E22000116,Isle of Wight,None,2009-01-01 00:00:00,None,live,E22,Community Safety Partnerships,CSP,...,81,0,2016-04-01 00:00:00,E22000001,E22000374,E22999999,Home Office,2009-01-01 00:00:00,2009-01-01 00:00:00,E22_CSP
3,E28_REGD,E28000146,Isle of Wight,None,2009-01-01 00:00:00,None,live,E28,Registration Districts,REGD,...,67,0,2013-01-17 00:00:00,E28000001,E28000219,E28999999,ONS,2009-01-01 00:00:00,2009-01-01 00:00:00,E28_REGD
4,E30_TTWA,E30000070,Isle of Wight,None,2009-01-01 00:00:00,None,live,E30,Travel to Work Areas,TTWA,...,206,6,2015-07-31 00:00:00,E30000004,E30000294,E30999999,ONS,2009-01-01 00:00:00,2009-01-01 00:00:00,E30_TTWA
5,E31_FRA,E31000021,Isle of Wight,None,2009-01-01 00:00:00,None,live,E31,Fire and Rescue Authorities,FRA,...,10,0,2016-08-01 00:00:00,E31000001,E31000047,E31999999,MHCLG,2009-01-01 00:00:00,2009-01-01 00:00:00,E31_FRA
6,E41_CMLAD,E41000046,Isle of Wight,None,2011-12-31 00:00:00,None,live,E41,Census Merged Local Authority Districts,CMLAD,...,0,0,None,E41000001,E41000324,E41999999,ONS,2013-04-01 00:00:00,2011-12-31 00:00:00,E41_CMLAD


In [49]:
tmp.columns

Index(['sheet', 'GEOGCD', 'GEOGNM', 'GEOGNMW', 'OPER_DATE', 'TERM_DATE',
       'STATUS', 'Entity code', 'Entity name', 'Entity abbreviation',
       'Entity theme', 'Entity coverage', 'Related entity codes', 'Status',
       'Number of live instances', 'Number of archived instances',
       'Number of cross-border instances', 'Date of last instance change',
       'Current code (first in range)', 'Current code (last in range)',
       'Reserved code (for CHD use)', 'Entity owner',
       'Date entity introduced on RGC', 'Entity start date', 'codeAbbrv'],
      dtype='object')

## More Example Queries

It would be useful to build up a list of sample queries.

In [74]:
q='''
SELECT GEOGCD, GEOGNM, `Entity code`, `Entity name`, `Entity abbreviation`, `Entity theme`, `Entity coverage` 
FROM codelist c JOIN metadata m
WHERE "GEOGNM"="{name}" AND codeAbbrv=sheet AND c.STATUS="live"'''

tmp = query_datasette_api(q.format(name = area_name),
                          "http://localhost:8001/onsgeocodes.json", dataframe=True)
tmp

[2019-02-28 16:47:45 +0000] - (sanic.access)[INFO][1:2]: GET http://localhost:8001/onsgeocodes.json?sql=%0ASELECT+GEOGCD%2C+GEOGNM%2C+%60Entity+code%60%2C+%60Entity+name%60%2C+%60Entity+abbreviation%60%2C+%60Entity+theme%60%2C+%60Entity+coverage%60+%0AFROM+codelist+c+JOIN+metadata+m%0AWHERE+%22GEOGNM%22%3D%22Isle+of+Wight%22+AND+codeAbbrv%3Dsheet+AND+c.STATUS%3D%22live%22  302 0


,GEOGCD,GEOGNM,Entity code,Entity name,Entity abbreviation,Entity theme,Entity coverage
0,E06000046,Isle of Wight,E06,Unitary Authorities,UA,Administrative,England
1,E14000762,Isle of Wight,E14,Westminster Parliamentary Constituencies,WPC,Electoral,England
2,E22000116,Isle of Wight,E22,Community Safety Partnerships,CSP,Other,England
3,E28000146,Isle of Wight,E28,Registration Districts,REGD,Other,England
4,E30000070,Isle of Wight,E30,Travel to Work Areas,TTWA,Other,England
5,E31000021,Isle of Wight,E31,Fire and Rescue Authorities,FRA,Other,England
6,E41000046,Isle of Wight,E41,Census Merged Local Authority Districts,CMLAD,Census,England


[2019-02-28 16:47:45 +0000] - (sanic.access)[INFO][1:2]: GET http://localhost:8001/onsgeocodes-93c9084.json?sql=%0ASELECT+GEOGCD%2C+GEOGNM%2C+%60Entity+code%60%2C+%60Entity+name%60%2C+%60Entity+abbreviation%60%2C+%60Entity+theme%60%2C+%60Entity+coverage%60+%0AFROM+codelist+c+JOIN+metadata+m%0AWHERE+%22GEOGNM%22%3D%22Isle+of+Wight%22+AND+codeAbbrv%3Dsheet+AND+c.STATUS%3D%22live%22  200 1171


## Lookup Tables

The raw code lookup is useful, but more useful is often the ability to find codes for areas bounded by other areas (for example, wards in a constituency).

A range of additional tables support such lookups. *It is left to further work to add in downloaders for building up a simple, comprehensive, lookup database.*

For example: http://geoportal.statistics.gov.uk/datasets?q=lookup

Also left to further work is to integrate data from other sources, such as the [NHS Digital administrative database](https://github.com/ouseful-datasupply/python-pandas-datareader-NHSDigital) and the [CQC register](https://github.com/ouseful-datasupply/cqc_uk).

### Output Area to LSOA to MSOA to Local Authority District (December 2017) Lookup with Area Classifications in Great Britain

http://geoportal.statistics.gov.uk/datasets/output-area-to-lsoa-to-msoa-to-local-authority-district-december-2017-lookup-with-area-classifications-in-great-britain

A lookup between Output Areas (OA), Lower Layer Super Output Areas (LSOA), Middle Layer Super Output Areas (LSOA) and local authority districts (LAD) including OA and LAD classifications as at 31 December 2017 in Great Britain.  (File size - 142 MB).  Field Names - OA11CD, OAC11CD, OAC11NM, LSOA11CD, LSOA11NM, SOAC11CD, SOAC11NM, MSOA11CD, MSOA11NM, LAD17CD, LAD17NM, LAC11CD, LAC11NM, RGN11CD, RGN11NM, CTRY11CD, CTRY11NM 

In [1]:
import pandas as pd

In [5]:
df_oa_lad = pd.read_csv('https://opendata.arcgis.com/datasets/fe6c55f0924b4734adf1cf7104a0173e_0.csv')

df_oa_lad[df_oa_lad['LAD17CD']=='E06000046'].head()

,OA11CD,OAC11CD,OAC11NM,LSOA11CD,LSOA11NM,SOAC11CD,SOAC11NM,MSOA11CD,MSOA11NM,LAD17CD,LAD17NM,LACCD,LACNM,RGN11CD,RGN11NM,CTRY11CD,CTRY11NM,FID
28002,E00087731,8c1,Ageing Industrious Workers,E01017370,Isle of Wight 018E,2d,Rural traits,E02003598,Isle of Wight 018,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,28003
28010,E00087478,7d1,Ageing Communities and Families,E01017322,Isle of Wight 011E,4a,Challenged white communities,E02003591,Isle of Wight 011,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,28011
28011,E00087500,7d1,Ageing Communities and Families,E01017325,Isle of Wight 009B,4b,Constrained renters,E02003589,Isle of Wight 009,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,28012
28012,E00087696,7d1,Ageing Communities and Families,E01017363,Isle of Wight 018A,5a,Ageing urban communities,E02003598,Isle of Wight 018,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,28013
28013,E00087291,6a4,Ageing in Suburbia,E01017284,Isle of Wight 010A,8a,Affluent communities,E02003590,Isle of Wight 010,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,28014


### Lower Layer Super Output Area (2011) to Ward (2018) Lookup in England and Wales v3


http://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-area-2011-to-ward-2018-lookup-in-england-and-wales-v3

This file is a best-fit lookup between 2011 lower layer super output areas, electoral wards/divisions and local authority districts in England and Wales as at 31 December 2018.  (File Size - 8 MB)  
Field Names - LSOA11CD, LSOA11NM, WD18CD, WD18NM, WD18NMW, LAD18CD, LAD18NM 

In [7]:
df_lsoa_ward = pd.read_csv('https://opendata.arcgis.com/datasets/8c05b84af48f4d25a2be35f1d984b883_0.csv')

df_lsoa_ward[df_lsoa_ward['LAD18CD']=='E06000046'].head()

,LSOA11CD,LSOA11NM,WD18CD,WD18NM,WD18NMW,LAD18CD,LAD18NM,FID
19635,E01017296,Isle of Wight 011C,E05008479,Arreton and Newchurch,NaN,E06000046,Isle of Wight,26636
19636,E01017324,Isle of Wight 015D,E05008479,Arreton and Newchurch,NaN,E06000046,Isle of Wight,26637
19637,E01017323,Isle of Wight 015C,E05008479,Arreton and Newchurch,NaN,E06000046,Isle of Wight,26638
19638,E01017295,Isle of Wight 009A,E05008482,Carisbrooke,NaN,E06000046,Isle of Wight,26639
19639,E01017294,Isle of Wight 013C,E05008482,Carisbrooke,NaN,E06000046,Isle of Wight,26640


### Ward to Westminster Parliamentary Constituency to Local Authority District (December 2018) Lookup in the United Kingdom

http://geoportal.statistics.gov.uk/datasets/ward-to-westminster-parliamentary-constituency-to-local-authority-district-december-2018-lookup-in-the-united-kingdom

This file is a lookup between electoral wards/divisions, Westminster parliamentary constituencies and local authority districts in the United Kingdom as at 31st December 2018.  (File Size - 2 MB)  
Field Names - WD18CD, WD18NM, PCON18CD, PCON18NM, LAD18CD, LAD18NM 

In [9]:
df_ward_wpc = pd.read_csv('https://opendata.arcgis.com/datasets/b7435625a4d442bcb331d610f16aacde_0.csv')

df_ward_wpc[df_ward_wpc['PCON18CD']=='E14000762'].head()

,WD18CD,WD18NM,PCON18CD,PCON18NM,LAD18CD,LAD18NM,FID
1452,E05008498,Newport East,E14000762,Isle of Wight,E06000046,Isle of Wight,4453
1465,E05008499,Newport North,E14000762,Isle of Wight,E06000046,Isle of Wight,4466
1471,E05008500,Newport South,E14000762,Isle of Wight,E06000046,Isle of Wight,4472
1477,E05008501,Newport West,E14000762,Isle of Wight,E06000046,Isle of Wight,4478
1483,E05008502,Parkhurst,E14000762,Isle of Wight,E06000046,Isle of Wight,4484


### Workplace Zone to Middle Layer Super Output Area to Local Authority District (December 2017) Lookup in Great Britain - Classifications

http://geoportal1-ons.opendata.arcgis.com/datasets/workplace-zone-to-middle-layer-super-output-area-to-local-authority-district-december-2017-lookup-in-great-britain-classifications

A lookup between workplace zones, middle layer super output areas and local authority districts with workplace zone and LAD classifications as at 31 December 2017 in Great Britain.  (File size - 3 MB) Field Names - WZ11CD, WZC11CD, WZC11NM, MSOA11CD, MSOA11NM, LAD17CD, LAD17NM, LACCD, LACNM, RGN11CD, RGN11NM, CTRY11CD, CTRY11NM

In [10]:
df_work_lad = pd.read_csv('https://opendata.arcgis.com/datasets/fde83309b6c14456846ca8fdece44a26_0.csv')

df_work_lad[df_work_lad['LAD17CD']=='E06000046'].head()

,WZ11CD,WZC11CD,WZC11NM,MSOA11CD,MSOA11NM,LAD17CD,LAD17NM,LACCD,LACNM,RGN11CD,RGN11NM,CTRY11CD,CTRY11NM,FID
36800,E33041130,F4,Traditional countryside,E02003593,Isle of Wight 013,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,38801
36801,E33041129,D1,Non-metropolitan suburban areas,E02003592,Isle of Wight 012,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,38802
36802,E33041128,F4,Traditional countryside,E02003592,Isle of Wight 012,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,38803
36803,E33041127,F1,Town fringe countryside,E02003592,Isle of Wight 012,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,38804
36806,E33041126,F4,Traditional countryside,E02003592,Isle of Wight 012,E06000046,Isle of Wight,3b1r,Ageing Coastal Living,E12000008,South East,E92000001,England,38807


### Postcode to Output Area to Lower Layer Super Output Area to Middle Layer Super Output Area to Local Authority District (November 2018) Lookup in the UK

https://ons.maps.arcgis.com/home/item.html?id=5d7199b9307d4876a3658814b72de7b0

A best-fit lookup between postcodes, frozen  2011 Census Output Areas (OA), Lower Layer Super Output Areas (LSOA), Middle Layer Super Output Areas (MSOA) and current local authority districts (LAD) as at November 2018 in the UK. Postcodes are best-fitted by plotting the location of the postcode's mean address into the areas of the output geographies. (File size 391 MB).
Field Names - PCD7, PCD8, PCDS, DOINTR, DOTERM, USERTYPE, OA11CD, LSOA11CD, MSOA11CD, LADCD, LSOA11NM, MSOA11NM, LADNM, LADNMW

(Need to click the link in the page?)